# Advanced Autopilot Systems: Waypoint Navigation and Autoland

This notebook explores the advanced autopilot features in simFlight, specifically:

1. Waypoint Navigation - Creating flight plans and navigating between waypoints
2. Autoland - Automatically approaching and landing at a runway

These features build upon the basic autopilot systems covered in previous notebooks, integrating multiple controllers to perform more complex flight tasks.

## 1. Setup

First, let's import the necessary modules and create our simulation environment:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

# Make plots interactive
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Import simFlight modules
from simflight.simulators import VirtualSimulator
from simflight.aircraft import Aircraft
from simflight.autopilot import AltitudeHold, HeadingHold, SpeedHold, Autoland
from simflight.navigation import Waypoint, FlightPlan, WaypointNavigation
from simflight.visualization import plot_simulation_results

In [ ]:
# Create the simulator with initial position
initial_state = {
    'altitude': 3000.0,         # 3000 feet
    'airspeed': 120.0,          # 120 knots
    'heading': 0.0,             # North
    'pitch': 0.0,
    'roll': 0.0,
    'latitude': 37.505,         # Near SFO
    'longitude': -122.300,
}

simulator = VirtualSimulator(initial_state=initial_state)
aircraft = Aircraft(simulator)

# Create autopilot controllers
altitude_hold = AltitudeHold(kp=0.01, ki=0.001, kd=0.05)
heading_hold = HeadingHold(kp=0.03, ki=0.001, kd=0.1)
speed_hold = SpeedHold(kp=0.05, ki=0.01, kd=0.0)

# Enable the controllers
altitude_hold.enable()
heading_hold.enable()
speed_hold.enable()

## 2. Waypoint Navigation

Waypoint navigation allows us to define a series of geographic points that the aircraft will follow automatically. Each waypoint specifies:

- Latitude and longitude (position)
- Altitude (height above sea level)
- Speed (desired airspeed)
- Optional name or identifier

Let's create a flight plan with several waypoints:

In [ ]:
# Create a flight plan
flight_plan = FlightPlan(name="SFO Approach Pattern")

# Add waypoints for a simple rectangular pattern
# Starting point (already at this position)
flight_plan.add_waypoint(Waypoint(
    latitude=37.505,
    longitude=-122.300,
    altitude=3000.0,
    speed=120.0,
    name="START"
))

# Turn to downwind
flight_plan.add_waypoint(Waypoint(
    latitude=37.495,
    longitude=-122.285,
    altitude=3000.0,
    speed=120.0,
    name="DOWNWIND_ENTRY"
))

# Downwind leg
flight_plan.add_waypoint(Waypoint(
    latitude=37.470,
    longitude=-122.265,
    altitude=2500.0,
    speed=110.0,
    name="DOWNWIND"
))

# Base leg turn
flight_plan.add_waypoint(Waypoint(
    latitude=37.455,
    longitude=-122.285,
    altitude=2000.0,
    speed=100.0,
    name="BASE_TURN"
))

# Final approach fix
flight_plan.add_waypoint(Waypoint(
    latitude=37.470,
    longitude=-122.305,
    altitude=1500.0,
    speed=90.0,
    name="FINAL_APPROACH"
))

# Runway threshold
runway_threshold_lat = 37.485
runway_threshold_lon = -122.325
runway_altitude = 0.0
runway_heading = 30.0

flight_plan.add_waypoint(Waypoint(
    latitude=runway_threshold_lat,
    longitude=runway_threshold_lon,
    altitude=runway_altitude + 50.0,  # Slightly above runway
    speed=80.0,
    name="RUNWAY_THRESHOLD"
))

# Display the flight plan
print(flight_plan)

Now, let's create a waypoint navigation system that will use our autopilot controllers to fly this route:

In [ ]:
# Create the waypoint navigation system
nav = WaypointNavigation(
    aircraft,
    flight_plan=flight_plan,
    heading_controller=heading_hold,
    altitude_controller=altitude_hold,
    speed_controller=speed_hold
)

# Enable the navigation system
nav.enable()

Now let's run a simulation where the aircraft follows the flight plan. We'll collect data at each step and visualize the flight path:

In [ ]:
# Run the simulation
simulation_steps = 1000
dt = 0.1  # seconds

# Lists to store position data for plotting
latitudes = []
longitudes = []
altitudes = []
airspeeds = []
waypoint_reached = []

# Store waypoint positions for plotting
wp_lats = [wp.latitude for wp in flight_plan.waypoints]
wp_lons = [wp.longitude for wp in flight_plan.waypoints]

for i in range(simulation_steps):
    # Apply navigation control
    nav.control_aircraft()
    
    # Run simulation step
    simulator.step()
    
    # Store data for plotting
    latitudes.append(aircraft.state['latitude'])
    longitudes.append(aircraft.state['longitude'])
    altitudes.append(aircraft.state['altitude'])
    airspeeds.append(aircraft.state['airspeed'])
    
    # Check if we've reached a waypoint
    if nav.update():
        waypoint_reached.append((
            aircraft.state['latitude'],
            aircraft.state['longitude']
        ))
    
    # If we've reached the final waypoint, stop the simulation
    if nav.flight_plan.current_waypoint_index == len(nav.flight_plan.waypoints) - 1:
        break

# Plot the flight path
plt.figure(figsize=(10, 8))
plt.plot(longitudes, latitudes, 'b-', label='Flight Path')
plt.plot(wp_lons, wp_lats, 'ro', label='Waypoints')

# Label the waypoints
for i, wp in enumerate(flight_plan.waypoints):
    plt.text(wp.longitude, wp.latitude, f"  {wp.name}", fontsize=9)

# Mark waypoints that were reached
if waypoint_reached:
    reached_lats, reached_lons = zip(*[(lat, lon) for lat, lon in waypoint_reached])
    plt.plot(reached_lons, reached_lats, 'go', label='Waypoint Reached')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flight Path with Waypoints')
plt.legend()
plt.grid(True)
plt.show()

# Plot altitude profile
plt.figure(figsize=(10, 4))
plt.plot(np.arange(len(altitudes)) * dt, altitudes)
plt.xlabel('Time (s)')
plt.ylabel('Altitude (ft)')
plt.title('Altitude Profile')
plt.grid(True)
plt.show()

# Plot airspeed profile
plt.figure(figsize=(10, 4))
plt.plot(np.arange(len(airspeeds)) * dt, airspeeds)
plt.xlabel('Time (s)')
plt.ylabel('Airspeed (knots)')
plt.title('Airspeed Profile')
plt.grid(True)
plt.show()

## 3. Autoland System

The autoland system takes over from waypoint navigation when the aircraft is on final approach. It manages:

1. **Glide Slope Control** - Maintains the correct descent angle to the runway
2. **Localizer Control** - Keeps the aircraft aligned with the runway centerline
3. **Airspeed Control** - Manages speed during approach and landing
4. **Flare Maneuver** - Reduces descent rate just before touchdown
5. **Rollout** - Keeps the aircraft on the runway after touchdown

Let's set up an autoland system for our approach:

In [ ]:
# Create the autoland system
autoland = Autoland(
    aircraft,
    target_approach_speed=80.0,   # Approach speed in knots
    target_glide_slope=3.0,       # 3-degree glide slope
    flare_altitude=30.0           # Begin flare at 30 feet
)

# Set runway parameters
autoland.set_runway(
    threshold_lat=runway_threshold_lat,
    threshold_lon=runway_threshold_lon,
    altitude=runway_altitude,
    heading=runway_heading
)

Now, let's reset our simulation and run a complete approach and landing sequence. We'll use waypoint navigation for the approach, then switch to autoland for the final landing phase:

In [ ]:
# Reset the simulator to the initial state
simulator = VirtualSimulator(initial_state=initial_state)
aircraft = Aircraft(simulator)

# Reset controllers
altitude_hold = AltitudeHold(kp=0.01, ki=0.001, kd=0.05)
heading_hold = HeadingHold(kp=0.03, ki=0.001, kd=0.1)
speed_hold = SpeedHold(kp=0.05, ki=0.01, kd=0.0)

altitude_hold.enable()
heading_hold.enable()
speed_hold.enable()

# Reset navigation
nav = WaypointNavigation(
    aircraft,
    flight_plan=flight_plan,
    heading_controller=heading_hold,
    altitude_controller=altitude_hold,
    speed_controller=speed_hold
)
nav.enable()

# Reset autoland
autoland = Autoland(
    aircraft,
    target_approach_speed=80.0,
    target_glide_slope=3.0,
    flare_altitude=30.0
)
autoland.set_runway(
    threshold_lat=runway_threshold_lat,
    threshold_lon=runway_threshold_lon,
    altitude=runway_altitude,
    heading=runway_heading
)

# Run the complete approach and landing simulation
simulation_steps = 2000
dt = 0.1  # seconds

# Lists to store data for plotting
latitudes = []
longitudes = []
altitudes = []
airspeeds = []
phases = []
current_phase = "Navigation"

for i in range(simulation_steps):
    # Store current state
    latitudes.append(aircraft.state['latitude'])
    longitudes.append(aircraft.state['longitude'])
    altitudes.append(aircraft.state['altitude'])
    airspeeds.append(aircraft.state['airspeed'])
    phases.append(current_phase)
    
    # Check if we should switch to autoland
    if nav.enabled and nav.flight_plan.current_waypoint_index == len(nav.flight_plan.waypoints) - 1:
        # We've reached the final waypoint, switch to autoland
        nav.disable()
        autoland.enable()
        current_phase = "Autoland: " + autoland.phase
        print(f"Switching to autoland at time {i*dt:.1f}s")
    
    # Update control based on active system
    if nav.enabled:
        nav.control_aircraft()
    elif autoland.enabled:
        autoland.update()
        if current_phase != "Autoland: " + autoland.phase:
            current_phase = "Autoland: " + autoland.phase
            print(f"Autoland phase change: {autoland.phase} at time {i*dt:.1f}s")
    
    # Run simulation step
    simulator.step()
    
    # Check if landing is complete
    if autoland.enabled and autoland.phase == 'rollout' and aircraft.state['ground_speed'] < 10:
        print(f"Landing complete at time {i*dt:.1f}s")
        break

# Plot the flight path with color-coded phases
plt.figure(figsize=(10, 8))

# Create a colormap for different phases
phase_types = sorted(list(set(phases)))
colors = plt.cm.viridis(np.linspace(0, 1, len(phase_types)))
phase_colors = {phase: color for phase, color in zip(phase_types, colors)}

# Plot each phase segment with different colors
for i, phase in enumerate(phase_types):
    mask = [p == phase for p in phases]
    if any(mask):
        plt.plot(
            [lon for lon, m in zip(longitudes, mask) if m],
            [lat for lat, m in zip(latitudes, mask) if m],
            '-',
            color=phase_colors[phase],
            label=phase
        )

# Plot waypoints
plt.plot(wp_lons, wp_lats, 'ro', label='Waypoints')

# Mark runway
runway_length = 0.02  # Approximate length in degrees
runway_end_lat = runway_threshold_lat + runway_length * np.sin(np.radians(runway_heading))
runway_end_lon = runway_threshold_lon + runway_length * np.cos(np.radians(runway_heading))
plt.plot([runway_threshold_lon, runway_end_lon], 
         [runway_threshold_lat, runway_end_lat], 
         'k-', linewidth=3, label='Runway')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Approach and Landing Path')
plt.legend()
plt.grid(True)
plt.show()

# Plot altitude profile
plt.figure(figsize=(12, 6))

# Plot altitude
plt.subplot(2, 1, 1)
times = np.arange(len(altitudes)) * dt
plt.plot(times, altitudes)

# Add phase transition markers
phase_changes = [i for i in range(1, len(phases)) if phases[i] != phases[i-1]]
for i in phase_changes:
    plt.axvline(x=times[i], color='r', linestyle='--', alpha=0.5)
    plt.text(times[i], max(altitudes), phases[i], rotation=90, verticalalignment='top')

plt.xlabel('Time (s)')
plt.ylabel('Altitude (ft)')
plt.title('Altitude Profile During Approach and Landing')
plt.grid(True)

# Plot airspeed
plt.subplot(2, 1, 2)
plt.plot(times, airspeeds)

# Add phase transition markers
for i in phase_changes:
    plt.axvline(x=times[i], color='r', linestyle='--', alpha=0.5)

plt.xlabel('Time (s)')
plt.ylabel('Airspeed (knots)')
plt.title('Airspeed Profile During Approach and Landing')
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. The Complete Autopilot System

In this notebook, we've explored how to create a complete autopilot system that can:

1. Follow a sequence of waypoints with specific altitude and speed targets
2. Automatically transition to an approach and landing sequence
3. Manage all phases of landing including approach, flare, touchdown, and rollout

This demonstrates how individual PID controllers can be combined into more complex systems to perform sophisticated flight tasks. The modular design allows for easy customization and extension to meet different requirements.

### Key Concepts Learned:

- **Flight Planning**: Defining waypoints and routes for navigation
- **Integrated Control Systems**: Combining multiple controllers for complex tasks
- **Phase-based Control Logic**: Different control strategies for different flight phases
- **Navigation Algorithms**: Great circle calculations for aviation navigation

These concepts form the foundation of real-world autopilot systems used in both commercial and general aviation.